In [1]:
from needed_files.quantumenvironment import QuantumEnvironment
from needed_files.helper_functions import load_agent_from_yaml_file
from needed_files.ppo import make_train_ppo
from needed_files.q_env_config import get_q_env_config
# from needed_files.q_env_config import q_env_config as gate_q_env_config
# from needed_files.qenvconfig import QEnvConfiguration

import os
import time
import cloudpickle

/Users/lukasvoss/anaconda3/envs/braket/lib/python3.10/site-packages/qiskit_dynamics/dispatch/backends/jax.py:34: UserWarning: The functionality in the perturbation module of Qiskit Dynamics requires a JAX version <= 0.4.6, due to a bug in JAX versions > 0.4.6. For versions 0.4.4, 0.4.5, and 0.4.6, using the perturbation module functionality requires setting os.environ['JAX_JIT_PJIT_API_MERGE'] = '0' before importing JAX or Dynamics.
  warnings.warn(
I0000 00:00:1704959435.246469       1 tfrt_cpu_pjrt_client.cc:349] TfrtCpuClient created.


In [3]:
type(QEnvConfiguration().q_env_configuration)

NameError: name 'QEnvConfiguration' is not defined

In [4]:
from braket.tracking import Tracker
from braket.jobs import hybrid_job, save_job_result
from braket.jobs.metrics import log_metric
from braket.aws import AwsDevice

from braket.aws import AwsSession
from braket.aws import AwsQuantumJob
from braket.jobs.local import LocalQuantumJob

from qiskit_braket_provider import BraketLocalBackend, AWSBraketProvider, AWSBraketBackend, AWSBraketJob

aws_session = AwsSession(default_bucket="amazon-braket-us-west-1-lukasvoss")

In [5]:
def calibrate_gate(num_total_updates):
    print("Job started!!!!!")

    braket_task_costs = Tracker().start()
    ppo_params, network_params  = load_agent_from_yaml_file(file_path='config_yamls/agent_config.yaml')
    agent_config = {**ppo_params, **network_params}
    
    provider = AWSBraketProvider()
    backend = provider.get_backend('SV1')
    # temp = QEnvConfiguration()
    # gate_q_env_config = temp.q_env_configuration

    gate_q_env_config = get_q_env_config()
    
    q_env = QuantumEnvironment(gate_q_env_config)
    q_env.backend = backend
    
    ppo_agent = make_train_ppo(agent_config, q_env)

    # training_results = ppo_agent(total_updates=hyperparams["num_updates"], print_debug=True, num_prints=40)
    training_results = ppo_agent(total_updates=num_total_updates, print_debug=False, num_prints=40)
    # save_job_result({ "measurement_counts": training_results })

    print("Job completed!!!!!")

    training_results['task_summary'] = braket_task_costs.quantum_tasks_statistics()
    training_results['estimated cost'] = float(
            braket_task_costs.qpu_tasks_cost() + braket_task_costs.simulator_tasks_cost()
        )

    return training_results

#### Check if function works as expected

In [6]:
%%time

training_results = calibrate_gate(num_total_updates=1)

Job started!!!!!
SparsePauliOp(['II', 'IZ', 'ZI', 'ZZ'],
              coeffs=[ 0.25+0.j, -0.25+0.j,  0.25+0.j, -0.25+0.j])


  0%|          | 0/1 [00:00<?, ?it/s]

SparsePauliOp(['II', 'IZ', 'YI', 'YZ'],
              coeffs=[ 0.25+0.j, -0.25+0.j, -0.25+0.j,  0.25+0.j])


100%|██████████| 1/1 [00:00<00:00,  4.00it/s]

Fidelity History: []
Job completed!!!!!
CPU times: user 788 ms, sys: 161 ms, total: 949 ms
Wall time: 7.23 s


### Use a decorator to make it a Hybrid Job

In [7]:
job_name = f'cx-gate-calibration-decorator-{str(int(time.time()))}'

In [8]:
os.getcwd()

'/Users/lukasvoss/Documents/Master Wirtschaftsphysik/Masterarbeit Yale-NUS CQT/Quantum_Optimal_Control/00_AWS/04_Tests/00_Hybrid_Job'

In [9]:
needed_modules = [
    'needed_files.quantumenvironment', 
    'needed_files.helper_functions', 
    'needed_files.ppo', 
    'needed_files.q_env_config',
    # 'needed_files.qenvconfig'
]


@hybrid_job(device=None, local=False, include_modules=needed_modules, input_data='config_yamls/agent_config.yaml', job_name=job_name)
def calibrate_gate_hybrid_job(num_total_updates):
    return calibrate_gate(num_total_updates)

In [10]:
job = calibrate_gate_hybrid_job(num_total_updates=10)

RuntimeError: Serialization failed for decorator hybrid job. If you are referencing an object from outside the function scope, either directly or through function parameters, try instantiating the object inside the decorated function instead.

In [ ]:
job.state()

In [ ]:
while job.state() not in job.TERMINAL_STATES:
    print(job.state())
    time.sleep(30)

print(job.state())
print(job.result())